# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
!pip install hvplot --upgrade
!pip install bokeh --upgrade
!pip install holoviews --upgrade
import hvplot
!pip install bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.1.1
    Uninstalling bokeh-3.1.1:
      Successfully uninstalled bokeh-3.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: holoviews
    Found existing installation: holoviews 1.15.4
    Uninstalling holoviews-1.15.4:
      Successfully uninstalled holoviews-1.15.4


In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
geoapify_key="2625ab641c2e4d9ab53e508aeea06f14"

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
import pandas as pd
import numpy as np
city_data_df = pd.read_csv("/content/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display
import matplotlib.pyplot as plt
!pip install geoviews
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng","Lat",geo = True,size="Humidity",tiles = "OSM",frame_width = 800,frame_height = 600)

# Display the map plot
map_plot_1

# Display the map
print(map_plot_1)
plt.show()

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.describe()
city_data_df1=city_data_df.copy()
city_data_df2=city_data_df1[(city_data_df1["Max Temp"]<=18.28)&(city_data_df1["Humidity"]<=75)&(city_data_df1["Cloudiness"]<=75)&(city_data_df1["Wind Speed"]<=3.46)]
# YOUR CODE HERE
city_data_df2=city_data_df2.dropna()
# Drop any rows with null values

# Display sample data
print(city_data_df2)
#city_data_df2=city_data_df2.dropna(inplace=True)

     City_ID             City      Lat       Lng  Max Temp  Humidity  \
2          2      new norfolk -42.7826  147.0587     11.72        58   
4          4          lanzhou  36.0564  103.7922     14.53        48   
6          6           albany  42.6001  -73.9662     10.36        74   
36        36          drochia  48.0350   27.8161     13.39        30   
43        43            ancud -41.8697  -73.8203     14.75        70   
90        90     saint george  37.1041 -113.5841     13.33        44   
96        96          zhangye  38.9342  100.4517     11.11        16   
101      101            heihe  50.2441  127.4902      2.32        49   
115      115       cedar city  37.6775 -113.0619     14.11        49   
141      141          brenham  30.1669  -96.3977     14.47        40   
153      153       lavrentiya  65.5833 -171.0000     -4.78        75   
158      158         almaznyy  48.0448   40.0450     10.39        47   
171      171         la ronge  55.1001 -105.2842      5.14      

### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df2[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head()
hotel_df["City"]

<ipython-input-18-8c684149a6ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


2          new norfolk
4              lanzhou
6               albany
36             drochia
43               ancud
90        saint george
96             zhangye
101              heihe
115         cedar city
141            brenham
153         lavrentiya
158           almaznyy
171           la ronge
199               asau
210              simao
268            chengde
280            jiuquan
318            harwich
324             vostok
331         swan river
336            dunedin
350          pingliang
359              korla
367      saint anthony
397            carlyle
402              seoul
434             broome
442            tieling
461             la paz
468            juegang
476            chifeng
506    sfantu gheorghe
522            xuchang
532         mandalgovi
535            emirdag
555             bulgan
560            changli
565            wyndham
Name: City, dtype: object

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

#Notes: JP helped me with this section.

Starting hotel search
new norfolk - nearest hotel: Woodbridge on the Derwent
lanzhou - nearest hotel: 西园宾馆
albany - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
ancud - nearest hotel: Cabañas
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
heihe - nearest hotel: Санкт-Петербург
cedar city - nearest hotel: Baymont Inn & Suites
brenham - nearest hotel: Ant Street Inn
lavrentiya - nearest hotel: Фортуна
almaznyy - nearest hotel: Зверево
la ronge - nearest hotel: La Ronge Hotel and Suites
asau - nearest hotel: Hotel Romanița
simao - nearest hotel: Marvelous Land Hotel
chengde - nearest hotel: 紫御国际假日酒店
jiuquan - nearest hotel: 民族宾馆
harwich - nearest hotel: Bottle Kiln
vostok - nearest hotel: No hotel found
swan river - nearest hotel: Swan River Super 8
dunedin - nearest hotel: Scenic Hotel Dunedin City
pingliang - nearest hotel: 平凉新都酒店
korla - nearest hotel: Silver Star Hotel
saint anthony - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
6,albany,US,42.6001,-73.9662,74,No hotel found
36,drochia,MD,48.0350,27.8161,30,Hotel Confort
43,ancud,CL,-41.8697,-73.8203,70,Cabañas
90,saint george,US,37.1041,-113.5841,44,"The Advenire, an Autograph Collection Hotel"
96,zhangye,CN,38.9342,100.4517,16,Zhangye Hotel 张掖饭店
101,heihe,CN,50.2441,127.4902,49,Санкт-Петербург
115,cedar city,US,37.6775,-113.0619,49,Baymont Inn & Suites
141,brenham,US,30.1669,-96.3977,40,Ant Street Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng","Lat",geo = True,size="Humidity",tiles = "OSM",frame_width = 800,frame_height = 600)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)